In [ ]:
#Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats
from sklearn.metrics import mutual_info_score
import scipy.stats as ss
from scipy.stats import chi2_contingency
from itertools import combinations

In [ ]:
data_NY = pd.read_csv('/content/NY_airbnb.csv')
# Mostrar la estructura del DataFrame
print(data_NY.info()) # Obtener información sobre los tipos de datos y la cantidad de valores no nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [ ]:
print(data_NY.isnull().sum())

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


### conclusión:
Variables con Null irelevantes `name` tiene 16,  `host_name` 21 null. No parece que sean relevantes a la hora del tratamiento de los datos, sin embargo, las variables  `last_reviews` con 10052 null y `reviews_per_month` 10052 null si que parece relevante para el tratamiento de los datos.

In [ ]:
# Descripción del conjunto de datos, estándard.
# Para variables numéricas y categóricas:
data_NY.describe(include="all")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,48879,4.889500e+04,48874,48895,48895,48895.000000,48895.000000,48895,48895.000000,48895.000000,48895.000000,38843,38843.000000,48895.000000,48895.000000
unique,NaN,47905,NaN,11452,5,221,NaN,NaN,3,NaN,NaN,NaN,1764,NaN,NaN,NaN
top,NaN,Hillside Hotel,NaN,Michael,Manhattan,Williamsburg,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2019-06-23,NaN,NaN,NaN
freq,NaN,18,NaN,417,21661,3920,NaN,NaN,25409,NaN,NaN,NaN,1413,NaN,NaN,NaN
mean,1.901714e+07,NaN,6.762001e+07,NaN,NaN,NaN,40.728949,-73.952170,NaN,152.720687,7.029962,23.274466,NaN,1.373221,7.143982,112.781327
std,1.098311e+07,NaN,7.861097e+07,NaN,NaN,NaN,0.054530,0.046157,NaN,240.154170,20.510550,44.550582,NaN,1.680442,32.952519,131.622289
min,2.539000e+03,NaN,2.438000e+03,NaN,NaN,NaN,40.499790,-74.244420,NaN,0.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000
25%,9.471945e+06,NaN,7.822033e+06,NaN,NaN,NaN,40.690100,-73.983070,NaN,69.000000,1.000000,1.000000,NaN,0.190000,1.000000,0.000000
50%,1.967728e+07,NaN,3.079382e+07,NaN,NaN,NaN,40.723070,-73.955680,NaN,106.000000,3.000000,5.000000,NaN,0.720000,1.000000,45.000000
75%,2.915218e+07,NaN,1.074344e+08,NaN,NaN,NaN,40.763115,-73.936275,NaN,175.000000,5.000000,24.000000,NaN,2.020000,2.000000,227.000000


In [ ]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).
# Buscar valores extraños: ver los valores únicos en cada feature.
# Creamos un DataFrame con dos columnas: 'feature' y 'n_values', donde 'n_values' es la lista de valores únicos de cada columna.

features = []
n_values = []

for col in data_NY.columns:
    features.append(col)
    n_values.append(data_NY[col].unique())

valores_unicos_data_NY = pd.DataFrame({'feature': features, 'n_values': n_values})

print(valores_unicos_data_NY)

                           feature  \
0                               id   
1                             name   
2                          host_id   
3                        host_name   
4              neighbourhood_group   
5                    neighbourhood   
6                         latitude   
7                        longitude   
8                        room_type   
9                            price   
10                  minimum_nights   
11               number_of_reviews   
12                     last_review   
13               reviews_per_month   
14  calculated_host_listings_count   
15                availability_365   

                                             n_values  
0   [2539, 2595, 3647, 3831, 5022, 5099, 5121, 517...  
1   [Clean & quiet apt home by the park, Skylit Mi...  
2   [2787, 2845, 4632, 4869, 7192, 7322, 7356, 896...  
3   [John, Jennifer, Elisabeth, LisaRoxanne, Laura...  
4   [Brooklyn, Manhattan, Queens, Staten Island, B...  
5   [Kensington, 

------

In [ ]:
data_SY = pd.read_csv('/content/sydney_airbnb.csv')
# Mostrar la estructura del DataFrame
print(data_SY.info()) # Obtener información sobre los tipos de datos y la cantidad de valores no nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36662 entries, 0 to 36661
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36662 non-null  int64  
 1   name                            36650 non-null  object 
 2   host_id                         36662 non-null  int64  
 3   host_name                       36656 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   36662 non-null  object 
 6   latitude                        36662 non-null  float64
 7   longitude                       36662 non-null  float64
 8   room_type                       36662 non-null  object 
 9   price                           36662 non-null  int64  
 10  minimum_nights                  36662 non-null  int64  
 11  number_of_reviews               36662 non-null  int64  
 12  last_review                     

In [ ]:
print(data_SY.isnull().sum())

id                                    0
name                                 12
host_id                               0
host_name                             6
neighbourhood_group               36662
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       11937
reviews_per_month                 11937
calculated_host_listings_count        0
availability_365                      0
dtype: int64


### conclusión:
Variables con Null irelevantes `name` tiene 12,  `host_name` 6 null. No parece que sean relevantes a la hora del tratamiento de los datos, sin embargo, las variables `neighbourhood_group` con 36662 null,  `last_reviews` con 11937 null y `reviews_per_month` 11937 null si que parece relevante para el tratamiento de los datos.

In [16]:
# Descripción del conjunto de datos, estándard.
# Para variables numéricas y categóricas:
data_SY.describe(include="all")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,3.666200e+04,36650,3.666200e+04,36656,0.0,36662,36662.000000,36662.000000,36662,36662.000000,36662.000000,36662.000000,24725,24725.000000,36662.000000,36662.000000
unique,NaN,35857,NaN,7874,NaN,38,NaN,NaN,3,NaN,NaN,NaN,1407,NaN,NaN,NaN
top,NaN,Home away from home,NaN,David,NaN,Sydney,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2018-12-01,NaN,NaN,NaN
freq,NaN,21,NaN,386,NaN,9241,NaN,NaN,22918,NaN,NaN,NaN,716,NaN,NaN,NaN
mean,1.852083e+07,NaN,6.561620e+07,NaN,NaN,NaN,-33.863112,151.204259,NaN,207.689597,5.096885,12.185178,NaN,1.139057,7.115106,97.387595
std,8.473884e+06,NaN,6.374533e+07,NaN,NaN,NaN,0.071501,0.083529,NaN,323.818795,15.664862,28.652548,NaN,1.536521,24.611284,122.969054
min,1.235100e+04,NaN,8.346000e+03,NaN,NaN,NaN,-34.135212,150.642903,NaN,0.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000
25%,1.170461e+07,NaN,1.506022e+07,NaN,NaN,NaN,-33.898444,151.179678,NaN,80.000000,1.000000,0.000000,NaN,0.140000,1.000000,0.000000
50%,2.027350e+07,NaN,4.134819e+07,NaN,NaN,NaN,-33.882486,151.215923,NaN,136.000000,2.000000,2.000000,NaN,0.490000,1.000000,36.000000
75%,2.524405e+07,NaN,1.053605e+08,NaN,NaN,NaN,-33.832072,151.261413,NaN,223.000000,5.000000,9.000000,NaN,1.520000,2.000000,165.000000


In [18]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).
# Buscar valores extraños: ver los valores únicos en cada feature.
# Creamos un DataFrame con dos columnas: 'feature' y 'n_values', donde 'n_values' es la lista de valores únicos de cada columna.

features = []
n_values = []

for col in data_NY.columns:
    features.append(col)
    n_values.append(data_SY[col].unique())

valores_unicos_data_SY = pd.DataFrame({'feature': features, 'n_values': n_values})

print(valores_unicos_data_SY)

                           feature  \
0                               id   
1                             name   
2                          host_id   
3                        host_name   
4              neighbourhood_group   
5                    neighbourhood   
6                         latitude   
7                        longitude   
8                        room_type   
9                            price   
10                  minimum_nights   
11               number_of_reviews   
12                     last_review   
13               reviews_per_month   
14  calculated_host_listings_count   
15                availability_365   

                                             n_values  
0   [12351, 14250, 15253, 20865, 26174, 38073, 393...  
1   [Sydney City & Harbour at the door, Manly Harb...  
2   [17061, 55948, 59850, 64282, 110561, 103476, 1...  
3   [Stuart, Heidi, Morag, Fiona, Amanda, Connie, ...  
4                                               [nan]  
5   [Sydney, Manl

------

In [ ]:
data_TK = pd.read_csv('/content/tokyo_airbnb.csv')
# Mostrar la estructura del DataFrame
print(data_TK.info()) # Obtener información sobre los tipos de datos y la cantidad de valores no nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11466 entries, 0 to 11465
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   11466 non-null  int64  
 1   name                 11466 non-null  object 
 2   host_id              11466 non-null  int64  
 3   host_name            11450 non-null  object 
 4   neighbourhood_group  0 non-null      float64
 5   neighbourhood        11466 non-null  object 
 6   latitude             11466 non-null  float64
 7   longitude            11466 non-null  float64
 8   room_type            11466 non-null  object 
 9   price                11466 non-null  int64  
 10  minimum_nights       11466 non-null  int64  
 11  number_of_reviews    11466 non-null  int64  
 12  last_review          9789 non-null   object 
 13  reviews_per_month    9789 non-null   float64
dtypes: float64(4), int64(5), object(5)
memory usage: 1.2+ MB
None


### ⚠ Importante
 faltan dos filas en este dataset: `calculated_host_listings_count` y `availability_365`

In [ ]:
print(data_TK.isnull().sum())

id                         0
name                       0
host_id                    0
host_name                 16
neighbourhood_group    11466
neighbourhood              0
latitude                   0
longitude                  0
room_type                  0
price                      0
minimum_nights             0
number_of_reviews          0
last_review             1677
reviews_per_month       1677
dtype: int64


### conclusión:
Variables con Null irelevantes `host_name` 16 null. No parece que sea relevante a la hora del tratamiento de los datos, sin embargo, las variables `neighbourhood_group` con 11466 null, `last_reviews` con 1677 null y `reviews_per_month` 1677 null si que parece relevante para el tratamiento de los datos.

In [17]:
# Descripción del conjunto de datos, estándard.
# Para variables numéricas y categóricas:
data_TK.describe(include="all")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
count,1.146600e+04,11466,1.146600e+04,11450,0.0,11466,11466.000000,11466.000000,11466,1.146600e+04,11466.000000,11466.000000,9789,9789.000000
unique,NaN,10810,NaN,2054,NaN,56,NaN,NaN,3,NaN,NaN,NaN,413,NaN
top,NaN,Near Ueno#Cozy Hotel#6min to Subway#40min airport,NaN,M-1 Tokyo,NaN,Shinjuku Ku,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2019-06-23,NaN
freq,NaN,22,NaN,133,NaN,1882,NaN,NaN,7463,NaN,NaN,NaN,395,NaN
mean,2.640545e+07,NaN,1.399776e+08,NaN,NaN,NaN,35.687159,139.734537,NaN,1.498470e+04,3.316239,26.028868,NaN,2.262540
std,7.565212e+06,NaN,8.064289e+07,NaN,NaN,NaN,0.217853,0.092771,NaN,3.327345e+04,7.766766,38.455449,NaN,1.744004
min,3.530300e+04,NaN,1.519770e+05,NaN,NaN,NaN,27.072330,139.130020,NaN,0.000000e+00,1.000000,0.000000,NaN,0.040000
25%,2.215251e+07,NaN,5.783171e+07,NaN,NaN,NaN,35.679620,139.698975,NaN,6.026000e+03,1.000000,2.000000,NaN,1.000000
50%,2.841873e+07,NaN,1.502326e+08,NaN,NaN,NaN,35.701910,139.729165,NaN,1.000700e+04,1.000000,12.000000,NaN,2.060000
75%,3.222416e+07,NaN,2.135896e+08,NaN,NaN,NaN,35.724215,139.784270,NaN,1.721600e+04,2.000000,32.000000,NaN,3.170000


In [20]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# asociados (n_values).
# Buscar valores extraños: ver los valores únicos en cada feature.
# Creamos un DataFrame con dos columnas: 'feature' y 'n_values', donde 'n_values' es la lista de valores únicos de cada columna.

features = []
n_values = []

for col in data_TK.columns:
    features.append(col)
    n_values.append(data_TK[col].unique())

valores_unicos_data_TK = pd.DataFrame({'feature': features, 'n_values': n_values})

print(valores_unicos_data_TK)

                feature                                           n_values
0                    id  [35303, 197677, 289597, 370759, 700253, 776070...
1                  name  [La Casa Gaienmae C Harajuku, Omotesando is ne...
2               host_id  [151977, 964081, 341577, 1573631, 801494, 4799...
3             host_name  [Miyuki, Yoshimi & Marek, Hide&Kei, Gilles,May...
4   neighbourhood_group                                              [nan]
5         neighbourhood  [Shibuya Ku, Sumida Ku, Nerima Ku, Setagaya Ku...
6              latitude  [35.67152, 35.717209999999994, 35.742670000000...
7             longitude  [139.71203, 139.82596, 139.6581, 139.65593, 13...
8             room_type       [Private room, Entire home/apt, Shared room]
9                 price  [4196, 10975, 6994, 3981, 6456, 9469, 5165, 96...
10       minimum_nights  [28, 3, 30, 29, 1, 5, 2, 4, 7, 35, 6, 14, 31, ...
11    number_of_reviews  [18, 156, 107, 99, 101, 215, 41, 90, 175, 85, ...
12          last_review  